In [22]:
import pyquasar as pq
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv

np.set_printoptions(precision=3, suppress=True)
%config InlineBackend.figure_format='retina'
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [23]:
plt.rcParams.update(
  {
    "figure.figsize": (6, 6),
    "axes.facecolor": "none",
    "font.size": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
  }
)

In [52]:
mesh = pq.Mesh.load("tetra.geo", refine_k=3)
# mesh = pq.Mesh.load("cube.geo", refine_k=0)
mesh

<Mesh object summary 
	Numeration: global
	Domains: [<MeshDomain object summary
	Material: steel
	Total elements number: 512
	Element type: Tetrahedron 4; Count: 512
	Boundary type: dirichlet; Tag: 1; Element type: Triangle 3; Count: 64.
	Boundary type: neumann; Tag: 2; Element type: Triangle 3; Count: 64.
	Boundary type: neumann; Tag: 3; Element type: Triangle 3; Count: 64.
	Boundary type: neumann; Tag: 4; Element type: Triangle 3; Count: 64.
>]>

In [53]:
def u(p, n):
  return p[..., 0] ** 2 - p[..., 1] ** 2
  return 2 * p[..., 0] + 2 * p[..., 1] + 2 * p[..., 2]
  return 2 * p[..., 0] + 3 * p[..., 1] + 5 * p[..., 2] - 4


def flow(p, n):
  return 2 * p[..., 0] * n[..., 0] - 2 * p[..., 1] * n[..., 1]
  return 2 * n[..., 0] + 2 * n[..., 1] + 2 * n[..., 2]
  return 2 * n[..., 0] + 3 * n[..., 1] + 5 * n[..., 2]

In [54]:
materials = {
  "dirichlet": u,
  "steel": {"neumann": flow, "steel": 0},
  "air": {"neumann": flow, "air": 0},
}

domains = [pq.FemDomain(domain) for domain in mesh.domains]
problem = pq.FemProblem(domains)
problem.assembly(materials)
problem.add_skeleton_projection(u, ["dirichlet"])
sol = problem.solve()
rel_err = np.linalg.norm(sol - u(mesh.domains[0].vertices, 0)) / np.linalg.norm(u(mesh.domains[0].vertices, 0))
print(f"Relative error: {rel_err:.2e}")

Relative error: 1.23e-02


In [55]:
# points = mesh.domains[0].vertices
# cells = np.array([np.insert(element, 0, 4) for element in mesh.domains[0].elements[0].node_tags])
# # cells1 = np.array([np.insert(element, 0, 4) for element in mesh.domains[0].elements[0].node_tags])
# # cells2 = np.array([np.insert(element, 0, 4) for element in mesh.domains[1].elements[0].node_tags])
# # cells = np.concatenate([cells1, cells2])
# celltypes = [pv.CellType.TETRA] * cells.shape[0]

# pv.set_jupyter_backend("trame")
# # pv.set_jupyter_backend("static")

# grid = pv.UnstructuredGrid()
# grid = pv.UnstructuredGrid(cells, celltypes, points)
# grid.point_data["u"] = sol
# grid.plot(show_edges=True)

In [56]:
points = np.array(
  [
    [0, 0, 0],
    # [0.1, 0.1, 0.1],
    # [0.5, 0, 0],
    # [0.5, 0, 0.5],
    [0, 1.0, 0],
    [1.0, 0.0, 0.0],
    [0, 0, 1]
  ]
)
proj = problem.project_into(points)

In [57]:
np.linalg.norm(proj @ sol - u(points, 0)) / np.linalg.norm(u(points, 0))

0.015829372790156244

In [58]:
proj_grad = problem.project_grad_into(points)
proj_grad[0] @ sol, proj_grad[1] @ sol, proj_grad[2] @ sol

(array([0.096, 0.124, 1.812, 0.124]),
 array([-0.12 , -1.864, -0.101, -0.125]),
 array([ 0.005, -0.   ,  0.03 , -0.005]))

In [59]:
grad_x = np.array([2 * p[0] for p in points])
grad_y = np.array([-2 * p[1] for p in points])
grad_z = 2 * np.ones(points.shape[0])
(
  np.linalg.norm(grad_x - proj_grad[0] @ sol) / np.linalg.norm(grad_x),
  np.linalg.norm(grad_y - proj_grad[1] @ sol) / np.linalg.norm(grad_y),
  # np.linalg.norm(grad_z - proj_grad[2] @ sol) / np.linalg.norm(grad_z),
)

(0.13689059996133146, 0.1210140184527428)